# Inverted Pendulum on a Drone

In [118]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import scipy
from sympy.physics.mechanics import *

### Standard Inverted Pendulum Dynamics

This is based on the Kinematics derivation of just the drone with the addition of the kinematics of the pendulum's angle.

Define states

In [119]:
# components of position (meters)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')

# yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# states of the pendulum
alpha, alpha_dot = sym.symbols('alpha, alpha_dot')

Define inputs

In [120]:
# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

Define parameters

m_pen is the mass of the pendulum and l_pen is the length of the pendulum

In [121]:
m, J_x, J_y, J_z, g, m_pen, l_pen = sym.symbols('m, J_x, J_y, J_z, g, m_pen, l_pen')

Create linear and angular velocity vectors (in coordinates of the body frame).

In [122]:
v_01in1 = sym.Matrix([v_x, v_y, v_z])
w_01in1 = sym.Matrix([w_x, w_y, w_z])

Create moment of inertia matrix (in coordinates of the body frame).

In [123]:
J_in1 = sym.diag(J_x, J_y, J_z)

## 2.2 Define kinematics of orientation

### 2.2.1 Rotation matrix in terms of yaw, pitch, roll angles

Define individual rotation matrices.

In [124]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation of the drone (i.e., of frame 1 in the coordinates of frame 0).

In [125]:
R_1in0 = Rz * Ry * Rx

In [126]:
R_1in0

⎡cos(ψ)⋅cos(θ)  sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)  sin(φ)⋅sin(ψ) + sin(θ)⋅c
⎢                                                                             
⎢sin(ψ)⋅cos(θ)  sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)  -sin(φ)⋅cos(ψ) + sin(ψ)⋅
⎢                                                                             
⎣   -sin(θ)                sin(φ)⋅cos(θ)                          cos(φ)⋅cos(θ

os(φ)⋅cos(ψ) ⎤
             ⎥
sin(θ)⋅cos(φ)⎥
             ⎥
)            ⎦

### 2.2.2 Map from angular velocity to angular rates

Recall that

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w_{0, 1}^{1}$$

for some matrix $N$. Here is how to compute that matrix for a ZYX (yaw, pitch, roll) Euler angle sequence.  First, we compute its inverse:

In [127]:
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([0, 0, 1]),
                              (Rx).T * sym.Matrix([0, 1, 0]),
                                       sym.Matrix([1, 0, 0]))

Then, we compute $N$ by taking the inverse of $N^{-1}$:

In [128]:
N = sym.simplify(Ninv.inv())

In [129]:
N

⎡      sin(φ)         cos(φ)    ⎤
⎢0     ──────         ──────    ⎥
⎢      cos(θ)         cos(θ)    ⎥
⎢                               ⎥
⎢0     cos(φ)         -sin(φ)   ⎥
⎢                               ⎥
⎣1  sin(φ)⋅tan(θ)  cos(φ)⋅tan(θ)⎦

## 2.3 Define equations of motion

This is where this document differs  widely from the standard equations of motion given in the class. The pendulum's affect on the motion of the drone is assumed to be negligible (is this true for the crazyflie? idk, we'll see)

### Deriving the Equations of Motion for the Pendulum

Generalized Co-ordinates: $x$, $z$ (in the world frame), $\theta$ (pitch) and $\alpha$ (angle of the drone from the world $z_0$)

In [130]:
dyn_o_x, dyn_o_z, dyn_alpha, dyn_theta = dynamicsymbols(r'o_x o_z \alpha \theta')
dyn_o_xd, dyn_o_zd, dyn_alphad, dyn_thetad = dynamicsymbols(r'o_x o_z \alpha \theta', 1)
dyn_o_xdd, dyn_o_zdd, dyn_alphadd, dyn_thetadd = dynamicsymbols(r'o_x o_z \alpha \theta', 2)
L_with_z = 1/2*m*(dyn_o_xd**2 + dyn_o_zd**2) + 1/2*J_y*(dyn_thetad**2) + 1/2*m_pen*(dyn_o_xd**2 + dyn_o_zd**2 + l_pen**2 * dyn_alphad**2 + 2*l_pen*dyn_alphad*(dyn_o_xd*sym.cos(dyn_alpha) - dyn_o_zd*sym.sin(dyn_alpha)) - m*g*dyn_o_z - m_pen*g*(dyn_o_z + l_pen*sym.cos(dyn_alpha)))

LM_with_z = LagrangesMethod(L_with_z, [dyn_o_x, dyn_o_z, dyn_alpha, dyn_theta])
mechanics_printing(pretty_print=True)
Equations_with_z = LM_with_z.form_lagranges_equations()
Equations_with_z

⎡                                                                             
⎢                                                             1.0⋅m⋅oₓ̈ + 0.5⋅
⎢                                                                             
⎢                                                                             
⎢                                               1.0⋅m⋅o_̈z - 0.5⋅mₚₑₙ⋅(-g⋅m - 
⎢                                                                             
⎢         ⎛      2                                                            
⎢0.5⋅mₚₑₙ⋅⎝2⋅lₚₑₙ ⋅\alp̈ha + 2⋅lₚₑₙ⋅(-sin(\alpha)⋅\alṗha⋅oₓ̇ - sin(\alpha)⋅o_
⎢                                                                             
⎣                                                                             

    ⎛                           2                                   ⎞         
mₚₑₙ⋅⎝- 2⋅lₚₑₙ⋅sin(\alpha)⋅\alṗha  + 2⋅lₚₑₙ⋅cos(\alpha)⋅\alp̈ha + 2⋅oₓ̈⎠     
                                                   

Solving the kinematic equation for $\ddot{\alpha}$:

In [131]:
alpha_ddot_soln = sym.solve(Equations_with_z[2], dyn_alphadd)
alpha_ddot_soln

Forces.

In [132]:
f_in1 = R_1in0.T * sym.Matrix([0, 0, -m * g]) + sym.Matrix([0, 0, f_z])

Torques.

In [133]:
tau_in1 = sym.Matrix([tau_x, tau_y, tau_z])

In [134]:
f_in1

⎡      g⋅m⋅sin(θ)       ⎤
⎢                       ⎥
⎢  -g⋅m⋅sin(φ)⋅cos(θ)   ⎥
⎢                       ⎥
⎣f_z - g⋅m⋅cos(φ)⋅cos(θ)⎦

In [135]:
tau_in1

⎡τₓ ⎤
⎢   ⎥
⎢τ_y⎥
⎢   ⎥
⎣τ_z⎦

Create equations of motion.

In [136]:
f_sym = sym.Matrix.vstack(R_1in0 * v_01in1,
                          N * w_01in1,
                          (1 / m) * (f_in1 - w_01in1.cross(m * v_01in1)),
                          J_in1.inv() * (tau_in1 - w_01in1.cross(J_in1 * w_01in1)),
                          sym.Matrix([dyn_alphad]),
                          sym.Matrix(alpha_ddot_soln)
                          )

Show equations of motion, which have the form

$$\dot{s} = f(s, i, p)$$

for states

$$
s = \begin{bmatrix} o_x \\ o_y \\ o_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \\ w_x \\ w_y \\ w_z \\ \alpha \\ \dot{\alpha} \end{bmatrix},
$$

inputs
$$
i = \begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \\ f_z \end{bmatrix},
$$

and parameters
$$
p = \begin{bmatrix} m \\ J_x \\ J_y \\ J_z \\ g \\ m_{pen} \\ l_{pen} \end{bmatrix}.
$$

In [137]:
f_sym

⎡vₓ⋅cos(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)) + v_z⋅(sin(φ)⋅s
⎢                                                                             
⎢vₓ⋅sin(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)) + v_z⋅(-sin(φ)⋅
⎢                                                                             
⎢                            -vₓ⋅sin(θ) + v_y⋅sin(φ)⋅cos(θ) + v_z⋅cos(φ)⋅cos(θ
⎢                                                                             
⎢                                          w_y⋅sin(φ)   w_z⋅cos(φ)            
⎢                                          ────────── + ──────────            
⎢                                            cos(θ)       cos(θ)              
⎢                                                                             
⎢                                          w_y⋅cos(φ) - w_z⋅sin(φ)            
⎢                                                                             
⎢                                wₓ + w_y⋅sin(φ)⋅tan

# 3. Derive state-space model

## 3.1 Choose equilibrium point

An equilibrium point of the nonlinear system is a choice of states $s_\text{eq}$ and inputs $i_\text{eq}$ - along with constant parameters $p_\text{eq}$ - for which

$$0 = f(s_\text{eq}, i_\text{eq}, p_\text{eq}).$$

Create a list of states, inputs, and parameters as symbolic variables.

In [138]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, alpha, alphad]
i = [tau_x, tau_y, tau_z, f_z]
p = [m, J_x, J_y, J_z, g, m_pen, l_pen]

Create a list of states to track as symbolic variables. These are states whose desired values will be specified by a client.

In [139]:
s_with_des = [o_x, o_y, o_z]

Create a function that evaluates $f(\cdot)$ at particular values of $s$, $i$, and $p$.

In [140]:
f = sym.lambdify(s + i + p, f_sym)

SyntaxError: unexpected character after line continuation character (<lambdifygenerated-2>, line 5)

In [ ]:
f

<function _lambdifygenerated(_Dummy_287, o_y, _Dummy_286, psi, _Dummy_288, phi, v_x, v_y, v_z, w_x, w_y, w_z, _Dummy_289, _Dummy_285, tau_x, tau_y, tau_z, f_z, m, J_x, J_y, J_z, g, m_pen, l_pen)>

Define constants.

In [ ]:
# Mass
m = 0.0312

# Principle moments of inertia
J_x = 1.57e-05
J_y = 1.60e-05
J_z = 2.05e-05

# Acceleration of gravity
g = 9.81

# Pendulum parameters
l_pen = 200e-3 # maybe m
density_pen = 8000 # kg/m3
diameter_pen = 1.2e-3 # m
m_pen = np.pi * (diameter_pen)**2/4 * l_pen * diameter_pen
# TODO: Actually measure the mass of the pendulum

Create a list of parameter values in the **same order** as the symbolic list. These are the parameter estimates we found in our experiments. They are not choices. (We use the subscript `_eq` to be consistent with what follows, and could say "parameter values *at equilibrium*," but don't be misled. These parameter values are *given* and are *constant* - again, they aren't choices.)

In [ ]:
p_eq = [m, J_x, J_y, J_z, g, l_pen, m_pen]

Create a list of state and input values at equilibrium in the **same order** as the symbolic lists.

In [ ]:
s_eq = [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
i_eq = [0., 0., 0., g*m]

Evaluate the equations of motion at the equilibrium point - if it actually *is* an equilibrium point, then the result should be an array of zeros:

In [ ]:
print(f(*s_eq, *i_eq, *p_eq))

NameError: name 'f' is not defined

Note that this equilibrium point would remain an equilibrium point for any choice of `o_x`, `o_y`, and `o_z` - that is one thing that allows the controller to track desired values of these variables.

## 3.2 Compute A and B

We want to find

$$
A = \frac{\partial f}{\partial s}\biggr\vert_{(s, i, p) = (s_\text{eq}, i_\text{eq}, p_\text{eq})}
\qquad\text{and}\qquad
B = \frac{\partial f}{\partial i}\biggr\vert_{(s, i, p) = (s_\text{eq}, i_\text{eq}, p_\text{eq})}.
$$

First, we compute each Jacobian (i.e., each matrix of partial derivatives) in symbolic form.

In [ ]:
A_sym = f_sym.jacobian(s)
B_sym = f_sym.jacobian(i)

Then, we create functions that allow us to evaluate these Jacobians at particular values of $s$, $i$, and $p$.

In [ ]:
A_num = sym.lambdify(s + i + p, A_sym)
B_num = sym.lambdify(s + i + p, B_sym)

Finally, we plug in our equilibrium point.

In [ ]:
A = A_num(*s_eq, *i_eq, *p_eq)
B = B_num(*s_eq, *i_eq, *p_eq)

Show $A$ (formatted nicely).

In [ ]:
A_str = np.array2string(A,
                        formatter={'float_kind': lambda x: f'{x:5.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'A = {A_str}')

A = [[ 0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00 -0.00  0.00 -0.00  0.00  1.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00]
     [ 0.00  0.00  0.00  0.00  0.00 -0.00  0.00  0.00  0.00  0.00  1.00 -0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  9.81  0.00  0.00  0.00 -0.00  0.00 -0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00 -9.81 -0.00  0.00  0.00  0.00  0.00 -0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00 -0.00  0.00 -0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]]


Show $B$ (formatted nicely).

In [ ]:
B_str = np.array2string(B,
                        formatter={'float_kind': lambda x: f'{x:10.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'B = {B_str}')

B = [[      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00      32.05]
     [  63694.27       0.00       0.00       0.00]
     [      0.00   62500.00       0.00       0.00]
     [      0.00       0.00   48780.49       0.00]]


The state-space system is described by

$$ \dot{x} = Ax + Bu $$

where

$$ x = s - s_\text{eq} $$

and

$$ u = i - i_\text{eq}. $$

Note that $A$ and $B$ would remain the same for any choice of `o_x`, `o_y`, and `o_z` - that is the other thing that allows the controller to track desired values of these variables.

# 4. Design method of power distribution

It should only be necessary to do this once.